In [1]:
class DFecomp:
    '''
    class DFecomp
    
    Compare DFe to PISCES
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 PISCESvar='Fe',
                 mapproj='rob',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 uplow=[0, 100],
                 cmap_extension='max',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 get_overview = False,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.PISCESvar = PISCESvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.uplow = uplow
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_PISCES import PISCESdata
        from Py_f2recom_toolbox import plt_Taylor_norm
        from Py_f2recom_toolbox import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        unitfesom = 'DFe [mmol m$^{-3}$]' # equals to mumol/L
        labelpisces = 'PISCES'
        unitpisces = 'DFe [mmol m$^{-3}$]'

        # load FESOM data -------------------------------------------------------------------------------------
        DFefesom = pf.get_data(resultpath, "DFe", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.mesh.path+'/'+self.runname+'.mesh.diag.nc'
        
        # load PISCES data  -------------------------------------------------------------------------------------
        PISCES_input = PISCESdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.PISCESvar, get_overview=False)
        pisces_int = PISCES_input.pisces_int    

        # apply sea mask to PISCES as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        pisces_int_ma = np.copy(pisces_int)
        pisces_int_ma[DFefesom == 0] = 0
        
        # plot PISCES and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            for i in np.arange(len(depth_array)-1):
                if depth_array[i+1] > np.max(PISCES_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(d,np.max(PISCES_input.layer_depths)))
                
                uplow = [depth_array[i], depth_array[i+1]]
                    
                DFefesom_mean = pf.layermean_data(DFefesom, mesh, uplow = uplow)
                DFepisces_mean = pf.layermean_data(pisces_int_ma, mesh, uplow = uplow)
                
                if(self.verbose):
                    print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nPISCES mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                        np.nanmean(DFefesom_mean),np.nanstd(DFefesom_mean),np.nanmin(DFefesom_mean),np.nanmax(DFefesom_mean),
                        np.nanmean(DFepisces_mean),np.nanstd(DFepisces_mean),np.nanmin(DFepisces_mean),np.nanmax(DFepisces_mean)))
                
                if plotting:
                    fig = plt.figure(figsize=(15,12), constrained_layout=False)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(0,3.1,.1)
                    f1 = pf.subplot(mesh, fig, m1, [DFefesom_mean],
                                levels = levels,
                                units=unitpisces, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, fig, m2, [DFepisces_mean], 
                                levels = levels,
                                units=unitpisces, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelpisces+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-3,3.1,0.1)
                    f3 = pf.subplot(mesh, fig, m3, [DFefesom_mean-DFepisces_mean], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitpisces, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - PISCES ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
                    m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                                size=30, weight='bold')
                    m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                                size=30, weight='bold')
                    
                    #fig.subplots_adjust(bottom=0.1)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DFe'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'DFe'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.pdf'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    plt.show(block=False)
                
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(DFepisces_mean)): print('WARNING: The interpolated PISCES field contains NaNs at depth')
                    if np.isnan(np.min(DFefesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Normalized Taylor Diagram for DFe'
                    plt_Taylor_comp(pisces_int_ma,DFefesom,mask=True,title=title, depth_array=depth_array, mesh=mesh,verbose = self.verbose)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                        dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                        bbox_inches='tight')
                    plt.show(block=False)
                    
                if output:
                    print('Only return non-layerwise output')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            if uplow[1] > np.max(PISCES_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(PISCES_input.layer_depths)))

            DFefesom_mean = pf.layermean_data(DFefesom, mesh, uplow = uplow)
            DFepisces_mean = pf.layermean_data(pisces_int_ma, mesh, uplow = uplow)
            
            if(self.verbose):
                print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nPISCES mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                np.nanmean(DFefesom_mean),np.nanstd(DFefesom_mean),np.nanmin(DFefesom_mean),np.nanmax(DFefesom_mean),
                np.nanmean(DFepisces_mean),np.nanstd(DFepisces_mean),np.nanmin(DFepisces_mean),np.nanmax(DFepisces_mean)))
                
        
            if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                            """
                            AB
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.01, 'bottom': 0.03}, 
                            subplot_kw=dict(projection=self.mapproj))
                
                
                m1 = axes['A']
                levels = np.arange(0,3.1,.1)
                f1 = pf.subplot(mesh, fig, m1, [DFefesom_mean],
                            levels = levels,
                            units=unitpisces, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n (0-{0} m)'.format(uplow[1]),
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [DFepisces_mean], 
                            levels = levels,
                            units=unitpisces, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelpisces+'\n (0-{0} m)'.format(uplow[1]),
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-3,3.1,0.1)
                f3 = pf.subplot(mesh, fig, m3, [DFefesom_mean-DFepisces_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitpisces, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - PISCES (0-{0} m)'.format(uplow[1]),
                            box=box,
                           )

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                
                #fig.subplots_adjust(bottom=0.3)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
                
            if Taylor:  
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(DFepisces_mean)): print('WARNING: The interpolated PISCES field contains NaNs at depth')
                if np.isnan(np.min(DFefesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Taylor Diagram for DFe \n(mean over depth, max = {0}-{1}m)'.format(uplow[0],uplow[1])
                plt_Taylor_norm(DFepisces_mean,DFefesom_mean,mask=True,title=title)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'DFe_PISCES_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)  
            
            if output:
                    self.DFefesom_mean = DFefesom_mean
                    self.DFepisces_mean   = DFepisces_mean